In [ ]:
import iris
import torch
import torch.optim as optim
import torch.nn as nn
import argparse
import os
import random
from torch.autograd import Variable
from torch.utils.data import DataLoader
import utils
import itertools
#import progressbar
import numpy as np
from scipy.ndimage.filters import gaussian_filter
import pdb
import re
import datetime
import time
import matplotlib.animation as manimation
#import cartopy.crs as ccrs
#import cartopy.feature as cf

import iris.plot as iplt
import iris.quickplot as qplt
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
batch_size = 3 #, type=int, help='batch size')
data_root = 'data' #', help='root directory for data')
#model_path = 'logs/lp/radar/model=dcgan128x128-rnn_size=256-predictor-posterior-prior-rnn_layers=2-1-1-n_past=3-n_future=7-lr=0.0020-g_dim=128-z_dim=10-last_frame_skip=True-beta=0.0001000/model4.pth'
#model_path = 'logs/lp/radar/model=vgg128x128-rnn_size=256-predictor-posterior-prior-rnn_layers=2-1-1-n_past=3-n_future=7-lr=0.0020-g_dim=128-z_dim=10-last_frame_skip=False-beta=0.0001000/model3.pth'
model_path = 'model131219.pth' #model4.pth'
log_dir = 'logs' #, help='directory to save generations to')
seed = 1 #', default=1, type=int, help='manual seed')
n_past = 3 #', type=int, default=3, help='number of frames to condition on')
n_future = 27 #', type=int, default=7, help='number of frames to predict')
num_threads = 0 #', type=int, default=0, help='number of data loading threads')
nsample = 100 #', type=int, default=100, help='number of samples')
N = 256 #', type=int, default=256, help='number of samples')

n_eval = n_past+n_future
max_step = n_eval

In [ ]:
print("Random Seed: ", seed)
random.seed(seed)
torch.manual_seed(seed)
#torch.cuda.manual_seed_all(opt.seed)
dtype = torch.FloatTensor

In [4]:
# ---------------- load the models  ----------------
tmp = torch.load(model_path, map_location='cpu')
frame_predictor = tmp['frame_predictor']
posterior = tmp['posterior']
prior = tmp['prior']
frame_predictor.eval()
prior.eval()
posterior.eval()
encoder = tmp['encoder']
decoder = tmp['decoder']
encoder.train()
decoder.train()
frame_predictor.batch_size = batch_size
posterior.batch_size = batch_size
prior.batch_size = batch_size
g_dim = tmp['opt'].g_dim
z_dim = tmp['opt'].z_dim
num_digits = tmp['opt'].num_digits

/home/h06/cbarth/anaconda3/lib/python3.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/h06/cbarth/anaconda3/lib/python3.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/h06/cbarth/anaconda3/lib/python3.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose2d' has changed. you can retrieve the original source c

In [5]:
# ---------------- set the options ----------------
dataset = tmp['opt'].dataset
last_frame_skip = tmp['opt'].last_frame_skip
channels = tmp['opt'].channels
image_width = tmp['opt'].image_width

In [6]:
# --------- load a dataset ------------------------------------

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]


def prep_data(files, filedir):

    # Regrid to a resolution x4 lower
    sample_points = [('projection_y_coordinate', np.linspace(-624500., 1546500., 543)),
                     ('projection_x_coordinate', np.linspace(-404500., 1318500., 431))]

    timeformat = "%Y%m%d%H%M"
    regex = re.compile("^/scratch/cbarth/phd/radar/(\d*)")
    
    #if filedir == 'train':
    #    regex = re.compile("^/nobackup/sccsb/radar/train/(\d*)")
    #elif filedir == 'test':
    #    regex = re.compile("^/nobackup/sccsb/radar/test/(\d*)")
    #elif filedir == 'may':
    #    regex = re.compile("^/nobackup/sccsb/radar/may/(\d*)")

    def gettimestamp(thestring):
        m = regex.search(thestring)
        return datetime.datetime.strptime(m.groups()[0], timeformat)

    # sort files by datetime
    sorted_files = sorted(files, key=gettimestamp)

    # only keep filenames where 10 consecutive files exist at 5 min intervals
    sorted_files = list(chunks(sorted_files, 30))
    for group in sorted_files:
        if len(group) < 30:
            sorted_files.remove(group)
        else:
            t0 = group[0].find('2018')
            dt1 = datetime.datetime.strptime(group[0][t0:t0+12], '%Y%m%d%H%M')
            t9 = group[29].find('2018')
            dt2 = datetime.datetime.strptime(group[29][t9:t9+12], '%Y%m%d%H%M')
            if (dt2-dt1 != datetime.timedelta(minutes=145)):
                print(dt2-dt1, 'remove files')
                sorted_files.remove(group)

    dataset = []
    for fn in sorted_files:
        cube = iris.load_cube(fn)
        cube = cube / 32.
        cube1 = cube.interpolate(sample_points, iris.analysis.Linear())
        data = cube1.data
        data = data[:, 160:288, 130:258] #focusing on a 128x128 grid box area over England

        # Set limit of large values - have asked Tim Darlington about these large values
        data[np.where(data < 0)] = 0.
        data[np.where(data > 32)] = 32.

        # Normalise data
        data = data / 32.

        dataset.append(data)

    #pdb.set_trace()
    # Convert to torch tensors
    dataset = dataset[:-1]
    tensor = torch.stack([torch.Tensor(i) for i in dataset])

    loader = DataLoader(tensor, #batch_size=1)
                        #num_workers=opt.data_threads,
                        batch_size=batch_size,
                        shuffle=True,
                        #drop_last=True,
                        pin_memory=True)
    return loader, cube1

In [7]:
val_dates = ['1109'] #0920'] #'0920'] #'1127'] #, '1109'] 

In [8]:
#files_v = [f'/nobackup/sccsb/radar/test/2018{mmdd}{h:02}{mi:02}_nimrod_ng_radar_rainrate_composite_1km_UK' \
#           for mi in range(0,60,5) for h in range(24) for mmdd in val_dates]
files_v = [f'/scratch/cbarth/phd/radar/2018{mmdd}{h:02}{mi:02}_nimrod_ng_radar_rainrate_composite_1km_UK' \
           for mi in range(0,60,5) for h in range(24) for mmdd in val_dates]
list_tst = []
for file in files_v:
    if os.path.isfile(file):
        list_tst.append(file)

In [9]:
test_loader, cube = prep_data(list_tst, 'test')

/home/h06/cbarth/anaconda3/lib/python3.7/site-packages/iris/fileformats/nimrod_load_rules.py:246: TranslationWarning: Vertical coord 5 not yet handled
  "".format(v_type), TranslationWarning)
/home/h06/cbarth/anaconda3/lib/python3.7/site-packages/iris/fileformats/nimrod_load_rules.py:246: TranslationWarning: Vertical coord 5 not yet handled
  "".format(v_type), TranslationWarning)
/home/h06/cbarth/anaconda3/lib/python3.7/site-packages/iris/fileformats/nimrod_load_rules.py:246: TranslationWarning: Vertical coord 5 not yet handled
  "".format(v_type), TranslationWarning)
/home/h06/cbarth/anaconda3/lib/python3.7/site-packages/iris/fileformats/nimrod_load_rules.py:246: TranslationWarning: Vertical coord 5 not yet handled
  "".format(v_type), TranslationWarning)
/home/h06/cbarth/anaconda3/lib/python3.7/site-packages/iris/fileformats/nimrod_load_rules.py:246: TranslationWarning: Vertical coord 5 not yet handled
  "".format(v_type), TranslationWarning)
/home/h06/cbarth/anaconda3/lib/python3.7

In [10]:
def get_testing_batch():
    while True:
        for sequence in test_loader: #.dataset:
            if np.shape(sequence)[0] == batch_size:
                batch = utils.normalize_data(dtype, sequence)
                yield batch

In [11]:
testing_batch_generator = get_testing_batch()

In [12]:
# --------- eval funtions ------------------------------------
def make_gifs(x, idx, name):
    # get approx posterior sample
    frame_predictor.hidden = frame_predictor.init_hidden()
    posterior.hidden = posterior.init_hidden()
    posterior_gen = []
    posterior_gen.append(x[0])
    x_in = x[0]
    for i in range(1, n_eval):
        #import pdb; pdb.set_trace()
        h = encoder(x_in)
        h_target = encoder(x[i])[0].detach()
        
        if last_frame_skip or i < n_past:
            h, skip = h
        else:
            h, _ = h
        h = h.detach()
        _, z_t, _= posterior(h_target) # take the mean
        if i < n_past:
            frame_predictor(torch.cat([h, z_t], 1)) 
            #posterior_gen.append(x[i])
            x_in = x[i]
            
            #x_in = x_in/x_in.max() # Added this to try renormalising the input frames too at each step
            posterior_gen.append(x_in)
            
        else:
            h_pred = frame_predictor(torch.cat([h, z_t], 1)).detach()
            x_in = decoder([h_pred, skip]).detach()
            
            #x_in = x_in/x_in.max()  # Added this to try renormalising the ouput at each step
            
            posterior_gen.append(x_in)

    #nsample = nsample
    ssim = np.zeros((batch_size, nsample, n_future))
    psnr = np.zeros((batch_size, nsample, n_future))
    #progress = progressbar.ProgressBar(max_value=nsample).start()
    all_gen = []
    for s in range(nsample):
        #progress.update(s+1)
        gen_seq = []
        gt_seq = []
        frame_predictor.hidden = frame_predictor.init_hidden()
        posterior.hidden = posterior.init_hidden()
        prior.hidden = prior.init_hidden()
        x_in = x[0]
        all_gen.append([])
        all_gen[s].append(x_in)
        for i in range(1, n_eval):
            h = encoder(x_in)
            if last_frame_skip or i < n_past:
                h, skip = h
            else:
                h, _ = h
            h = h.detach()
            if i < n_past:
                h_target = encoder(x[i])[0].detach()
                z_t, _, _ = posterior(h_target)
                prior(h)
                frame_predictor(torch.cat([h, z_t], 1))
                x_in = x[i]
                
                #x_in = x_in/x_in.max() # Added this to try renormalising the input frames too at each step
                
                all_gen[s].append(x_in)
            else:
                z_t, _, _ = prior(h)
                h = frame_predictor(torch.cat([h, z_t], 1)).detach()
                x_in = decoder([h, skip]).detach()
                      
                ## Added these lines to make prediction better match intensity of truth
                ##x_in = x_in.exp()
                #sum_x = x[i].sum()
                ##print('sum_x = ', sum_x)
                ##set values less than 20% of max to 0
                ##x_in[np.where(x_in < 0.2*x_in.max())] = 0
                #sum_pred = x_in.sum()
                ##print('sum_pred = ', sum_pred)
                #x_in = x_in*(sum_x/sum_pred)
                ##print('sum_x_new = ', x_in.sum())
                
                #x_in = x_in/x_in.max()  # Added this to try renormalising the output at each step
                
                gen_seq.append(x_in.data.cpu().numpy())
                gt_seq.append(x[i].data.cpu().numpy())
                all_gen[s].append(x_in)
        _, ssim[:, s, :], psnr[:, s, :] = utils.eval_seq(gt_seq, gen_seq)
        
    return(ssim, x, posterior_gen, all_gen) 

In [13]:
# Create cubes of right sizes
truth_cube = cube[:, 160:288, 130:258] 
pred_cube = cube[:, 160:288, 130:258] 
pred_cube1 = cube[:, 160:288, 130:258] 
pred_cube2 = cube[:, 160:288, 130:258] 
truth_cube *= 32.
pred_cube *= 32.
pred_cube1 *= 32.
pred_cube2 *= 32.

In [17]:
for i in range(0, N, batch_size):
    print(i)
    # plot test
    test_x = next(testing_batch_generator)
    ssim, x, posterior_gen, all_gen = make_gifs(test_x, i, 'test')
    for batch_number in range(batch_size):
        mean_ssim = np.mean(ssim[0], 1)
        ordered = np.argsort(mean_ssim)
        rand_sidx = [np.random.randint(nsample) for s in range(3)]
        sidx = ordered[-1]
        for t in range(n_eval):
            print('time = ', t)
            truth_cube.data[t] = x[t][batch_number][0].detach().numpy() *32.
            pred_cube.data[t] = all_gen[sidx][t][batch_number][0].detach().numpy() *32.
            pred_cube1.data[t] = all_gen[1][t][batch_number][0].detach().numpy() *32.  #random sample - choose index
            pred_cube2.data[t] = all_gen[8][t][batch_number][0].detach().numpy() *32.  #random sample - choose index

            truth_cube.units = 'm/s'
            pred_cube.units = 'm/s'
            pred_cube1.units = 'm/s'
            pred_cube2.units = 'm/s'
                        
            #sum_truth = truth_cube.data[t].sum()
            #sum_pred = pred_cube.data[t].sum()
            ##pred_cube = pred_cube * (sum_truth / sum_pred)
            #pred_cube.units = 'm/s'
            #print('pred sum = ', pred_cube.data[t].sum())
            #print('truth sum = ', truth_cube.data[t].sum())
            
            fig = plt.figure(figsize=(20,20))
            ax = fig.add_subplot(2,2,1)
            #cf = qplt.contourf(truth_cube[t], cmap=plt.cm.Blues) #, vmin=0, vmax=1) #32)
            cf = qplt.contourf(truth_cube[t], vmin=0, vmax=16) #truth_cube.data[t].max())
            plt.gca().coastlines('50m', color='white')
            plt.title('Radar rain rate')
            
            ax = fig.add_subplot(2,2,2)
            cf = qplt.contourf(pred_cube[t], vmin=0, vmax=16) #truth_cube.data[t].max())
            plt.gca().coastlines('50m', color='white')
            plt.title('Best SSIM prediction')
            
            ax = fig.add_subplot(2,2,3)
            cf = qplt.contourf(pred_cube1[t], vmin=0, vmax=16) #truth_cube.data[t].max())
            plt.gca().coastlines('50m', color='white')
            plt.title('Random sample prediction 1')
            
            ax = fig.add_subplot(2,2,4)
            cf = qplt.contourf(pred_cube2[t], vmin=0, vmax=16) #truth_cube.data[t].max())
            plt.gca().coastlines('50m', color='white')
            plt.title('Random sample prediction 2')
            
            plt.suptitle('T+{}'.format(t), fontsize=16)
            plt.savefig('comparison{}_{}_im{}.png'.format(i, batch_number, t))
            plt.close()

0
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
3


KeyboardInterrupt: 

Testing animation within the code

In [18]:
for i in range(0, N, batch_size):
    print(i)
    # plot test
    test_x = next(testing_batch_generator)
    ssim, x, posterior_gen, all_gen = make_gifs(test_x, i, 'test')
    for batch_number in range(batch_size):
        mean_ssim = np.mean(ssim[0], 1)
        ordered = np.argsort(mean_ssim)
        rand_sidx = [np.random.randint(nsample) for s in range(3)]
        sidx = ordered[-1]
        
        # Set up video configuration
        FFMpegWriter = manimation.writers['ffmpeg']
        metadata = dict(artist='Matplotlib')
        writer = FFMpegWriter(fps=2, metadata=metadata)
        # Configure plots
        fig = plt.figure(figsize=(20, 10))

        # Set video filename
        filename = "radar_animation2_{}_{}.mp4".format(i, batch_number)

        # Create plot frames
        with writer.saving(fig, filename, 300):
        
            for t in range(n_eval):
                print('time = ', t)
                truth_cube.data[t] = x[t][batch_number][0].detach().numpy() *32.
                pred_cube.data[t] = all_gen[sidx][t][batch_number][0].detach().numpy() *32.
                #pred_cube.data[t] = all_gen[1][t][batch_number][0].detach().numpy() *32.  #random sample - choose index
                pred_cube1.data[t] = all_gen[1][t][batch_number][0].detach().numpy() *32.  #random sample - choose index
                pred_cube2.data[t] = all_gen[8][t][batch_number][0].detach().numpy() *32.  #random sample - choose index

                truth_cube.units = 'm/s'
                pred_cube.units = 'm/s'
                pred_cube1.units = 'm/s'
                pred_cube2.units = 'm/s'
                #truth_cube.units = 'm/s'
                        
                #sum_truth = truth_cube.data[t].sum()
                #sum_pred = pred_cube.data[t].sum()
                #pred_cube = pred_cube * (sum_truth / sum_pred)
                #pred_cube.units = 'm/s'

                ax = fig.add_subplot(2,2,1)
                iplt.contourf(truth_cube[t], vmin=0, vmax=16)
                plt.gca().coastlines('50m', color='white')
                plt.title('Radar')
                ax = fig.add_subplot(2,2,2)
                iplt.contourf(pred_cube[t], vmin=0, vmax=16)
                plt.gca().coastlines('50m', color='white')
                plt.title('Best SSIM')
                ax = fig.add_subplot(2,2,3)
                iplt.contourf(pred_cube1[t], vmin=0, vmax=16)
                plt.gca().coastlines('50m', color='white')
                plt.title('Sample 1')
                ax = fig.add_subplot(2,2,4)
                iplt.contourf(pred_cube2[t], vmin=0, vmax=16)
                plt.gca().coastlines('50m', color='white')
                plt.title('Sample 2')
                
                fig.suptitle('T+{:02d}'.format(t), fontsize=16)

                # Save frame
                writer.grab_frame()
        plt.close()

0
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
3
time =  0
time =  1
time =  2
time =

time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
27
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time

time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
54
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time = 

time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
78
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time

time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
105
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =

...

time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
156
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =

time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
180
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
tim

time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
207
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =

time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
231
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
tim

time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29
time =  0
time =  1
time =  2
time =  3
time =  4
time =  5
time =  6
time =  7
time =  8
time =  9
time =  10
time =  11
time =  12
time =  13
time =  14
time =  15
time =  16
time =  17
time =  18
time =  19
time =  20
time =  21
time =  22
time =  23
time =  24
time =  25
time =  26
time =  27
time =  28
time =  29


In [ ]:
print(truth_cube)

In [ ]:
#        cube_iter = truth_cube.slices(('time'))
#        ani = iris.experimental.animate.animate(cube_iter, qplt.contourf)
#        plt.show() 
#        pdb.set_trace()
        
#        # Generate grid for plotting
#        x = linspace(0, 128)
#        y = linspace(0, 128)
#        x,y = meshgrid(x,y)

#        fig = plt.figure()
#        ax = plt.axes(xlim=(0, 128), ylim=(0, 128))  
#        plt.xlabel(r'x')
#        plt.ylabel(r'y')

#        # animation function
#        def animate(i): 
#            z = var[i,:,0,:].T
#            cont = plt.contourf(x, y, z, 25)
#            if (tslice == 0):
#                plt.title(r't = %1.2e' % t[i] )
#            else:
#                plt.title(r't = %i' % i)

#            return cont

#anim = animation.FuncAnimation(fig, animate, frames=Nt)
#anim.save('animation.mp4')
       
#import matplotlib.animation as animation
#data_gen.t = 0

# create a figure with two subplots
#fig, (ax1, ax2) = plt.subplots(2,1)
#ax1 = plt.axes(projection=ccrs.PlateCarree())
#ax2 = plt.axes(projection=ccrs.PlateCarree())

## intialize two line objects (one in each axes)
#subplot1, = ax1.contourf()
#subplot2, = ax2.contourf()
#plots = [subplot1, subplot2]

# the same axes initalizations as before (just now we do it for both of them)
#for ax in [ax1, ax2]:
#    ax.coastlines()

## initialize the data arrays 
#xdata, y1data, y2data = [], [], []

#ani = animation.FuncAnimation(fig, run, data_gen, blit=True, interval=10,
#    repeat=False)
#plt.show()

#        fig = plt.figure(figsize=(20,10))
#        ax = fig.add_subplot(1,2,1)
#        #cf = qplt.contourf(truth_cube[t], cmap=plt.cm.Blues) #, vmin=0, vmax=1) #32)
#        cf = qplt.contourf(truth_cube[t], vmin=0, vmax=10) #truth_cube.data[t].max())
#        plt.gca().coastlines('50m', color='white')
#        plt.title('Radar rain rate t+{}'.format(t))
#        ax = fig.add_subplot(1,2,2)
#        cf = qplt.contourf(pred_cube[t], vmin=0, vmax=10) #truth_cube.data[t].max())
#        plt.gca().coastlines('50m', color='white')
#        plt.title('Model rain rate prediction t+{}'.format(t))
#        plt.savefig('comparison{}_{}_im{}.png'.format(i, batch_number, t))
#        plt.close()

In [ ]:
import matplotlib.animation as manimation
import cartopy.crs as ccrs
import cartopy.feature as cf
import iris.plot as iplt

step_size = 30

# Set up video configuration
FFMpegWriter = manimation.writers['ffmpeg']
metadata = dict(artist='Matplotlib')
writer = FFMpegWriter(fps=2, metadata=metadata)
# Configure plots
fig = plt.figure(figsize=(20, 10))

# Set video filename
filename = "glm_animation_{}.mp4".format(i)

# Create plot frames
with writer.saving(fig, filename, 300):

    # Create one frame per [step_size] minutes
    for t in np.arange(step_size):
        ax = fig.add_subplot(1,2,1)
        iplt.contourf(truth_cube[t], vmin=0, vmax=10)
        plt.gca().coastlines('50m', color='white')
        plt.title('Radar rain rate')
        ax = fig.add_subplot(1,2,2)
        iplt.contourf(pred_cube[t], vmin=0, vmax=10)
        plt.gca().coastlines('50m', color='white')
        plt.title('Model rain rate prediction')
        fig.suptitle('T+{:02d}'.format(t), fontsize=16)

        # Save frame
        writer.grab_frame()
plt.close()